In [57]:
import word2vec as w2v
import gensim
import os
import csv
import nltk

In [71]:
porter = nltk.PorterStemmer()

def clean_token(w):
    w = porter.stem(''.join([a.lower() for a in w if a.isalnum()]))
    return w
    
def clean_phrase(s):
    s = s.split()
    s = ' '.join([clean_token(w) for w in s])
    return s

clean_phrase("Testing how\nthis thing splits\twhitespace")

u'test how thi thing split whitespac'

In [2]:
# concat court cases to train word2vec
casepath = 'CourtCases'

In [73]:
# Build clean file for Word2Vec training
fns = [fn for fn in os.listdir(casepath) if fn[0] != '.']
with open('clean_cases.txt', 'w') as catfile:
    for i, fn in enumerate(fns):
        with open(os.path.join(casepath, fn)) as infile:
            s = clean_phrase(infile.read())+'\n'
            catfile.write(s)
        if i % 1000 == 0:
            print "Read %d files" % i

Read 0 files
Read 1000 files
Read 2000 files
Read 3000 files
Read 4000 files
Read 5000 files
Read 6000 files
Read 7000 files
Read 8000 files
Read 9000 files
Read 10000 files
Read 11000 files
Read 12000 files
Read 13000 files
Read 14000 files
Read 15000 files
Read 16000 files
Read 17000 files
Read 18000 files
Read 19000 files
Read 20000 files
Read 21000 files
Read 22000 files
Read 23000 files
Read 24000 files
Read 25000 files
Read 26000 files
Read 27000 files
Read 28000 files
Read 29000 files
Read 30000 files
Read 31000 files
Read 32000 files
Read 33000 files
Read 34000 files
Read 35000 files
Read 36000 files
Read 37000 files
Read 38000 files
Read 39000 files
Read 40000 files
Read 41000 files
Read 42000 files
Read 43000 files
Read 44000 files
Read 45000 files
Read 46000 files
Read 47000 files
Read 48000 files
Read 49000 files
Read 50000 files
Read 51000 files
Read 52000 files
Read 53000 files
Read 54000 files
Read 55000 files
Read 56000 files
Read 57000 files
Read 58000 files
Read 59000

In [77]:
model = w2v.word2vec('clean_cases.txt', 'cases_w2v.bin', size=150, window=15)
model = gensim.models.Word2Vec.load_word2vec_format('cases_w2v.bin', binary=True)
# model = gensim.models.Word2Vec.load_word2vec_format('cases.bin', binary=True, unicode_errors='ignore')
# Forgot to clean text. This should improve performance.
# Edit: Cleaned for alphanum and lemmatized.

In [107]:
model.most_similar(positive='bond', negative=['guaranti', 'guarante'])

[(u'c', 0.6478906869888306),
 (u'e', 0.6005458831787109),
 (u'k', 0.5990673303604126),
 (u'h', 0.5908220410346985),
 (u'g', 0.5875924825668335),
 (u'b2a', 0.5843648910522461),
 (u'60054511', 0.5788819789886475),
 (u'b1', 0.5507212281227112),
 (u'f', 0.5494710206985474),
 (u'l', 0.5340996980667114)]

In [78]:
with open('inputData.csv', 'r') as infile:
    reader = csv.reader(infile)
    data = [row for row in reader]

with open('outputData.csv', 'r') as testfile:
    reader = csv.reader(testfile)
    test = [row for row in reader]

questions = [d[0] for d in data]
answers = [d[1:] for d in data]

validation = [t[1] for t in test]

In [83]:
# Similarity between sentences

def similarity(q, a):
    score = 0.0
    no_count = 0
    # Get inner product
    # score = sum([w*q for (w,q) in zip(wq,wa)])
    for wq in q.split():
        for wa in a.split():
            try: # Shouldn't happen anymore
                score += model.similarity(wq, wa)
            except: # Shouldn't happen anymore
                no_count += 1
    score /= ((len(q)*len(a)) - no_count)
    return score

def rank_answers(q, alist):
    # Find answer that maximizes inner product of embeddings
    # Credit: Miyyer
    simlist = []
    for a in alist:
        simlist.append(similarity(q,a))
    #return [a for (a, s) in sorted(zip(alist, simlist))]
    return alist[simlist.index(max(simlist))]

In [84]:
top_answer = [rank_answers(q,a) for q,a in zip(questions,answers)]

In [85]:
similarity(questions[0], answers[0][0])

0.0018186942844012264

In [86]:
correct = [int(i == j) for (i,j) in zip(top_answer, validation)]
(sum(correct)*1.0)/len(validation)

0.33666666666666667

In [48]:
rank_answers(questions[0],answers[0])

'Section 502(b)(6) caps a landlord\'s claim in bankruptcy for damages resulting from the termination of a real property lease.16 Under \xc2\xa7 502(b)(6), a landlord-creditor is entitled to rent reserved from the greater of (1) one lease year or (2) fifteen percent, not to exceed three years, of the remaining lease term. The cap operates from the earlier of the petition filing date or "the date on which [the] lessor repossessed or the lessee surrendered, the leased property." The landlord also retains a claim for any unpaid rent due under such lease prior to the earlier of those dates. This language reflects Congress\'s intent to limit lease termination claims to prevent landlords from receiving a windfall over other creditors. See H.R.Rep. No. 95-595, at 353 (1977), reprinted in 1978 U.S.C.C.A.N. 5963, 6309 ("[The cap] limits the damages allowable to a landlord of the debtor.... It is designed to compensate the landlord for his loss while not permitting a claim so large (based on a lo